In [ ]:
from odc.stac import load as stac_load
from pystac_client import Client
import geopandas as gpd
from shapely.geometry import mapping

aoi = gpd.read_file("CODF_Chishan.gpkg", layer="CODF_high_lines")
aoi_geom = aoi[aoi["BU ID"]=="BO.004.03"].geometry.iloc[0]

catalog = Client.open("/projectnb/modislc/users/chishan/data/hansen_stac_catalog/catalog.json")
items = catalog.search(bbox=aoi_geom.bounds, query={"layer": {"eq": "treecover2000"}}).get_all_items()

ds = stac_load(items, bands=["treecover2000"], chunks={"x":2048, "y":2048})
clipped = ds.rio.clip([mapping(aoi_geom)], aoi.crs)
clipped.treecover2000.plot()


In [ ]:
from odc.stac import load as stac_load
from pystac_client import Client
from planetary_computer import sign
import geopandas as gpd

# 1. 连接 MPC 的 STAC API
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

# 2. 定义 AOI
aoi = gpd.read_file("/path/to/your_AOI.gpkg")
geom = aoi.geometry.iloc[0]
bbox = geom.bounds

# 3. 查询数据
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime="2020-06-01/2020-06-30",
    query={"eo:cloud_cover": {"lt": 10}}
)

items = list(search.get_items())

# 4. MPC 需要签名 URL (Planetary Computer SAS tokens)
#    用 planetary_computer.sign 自动完成
for item in items:
    sign(item)

# 5. 直接加载为 xarray DataCube
ds = stac_load(items, bands=["B04", "B08"], chunks={"x": 2048, "y": 2048})


In [ ]:
from odc.stac import load as stac_load
from pystac_client import Client
from planetary_computer import sign
import dask
from dask.distributed import Client as DaskClient
import geopandas as gpd

# 启动 Dask（HPC 可用 SLURM 分配节点）
dask_client = DaskClient()

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
aoi = gpd.read_file("/project_data/CODF/CODF_Chishan.gpkg", layer="CODF_high_lines")
geom = aoi[aoi["BU ID"]=="BO.004.03"].geometry.iloc[0]

search = catalog.search(
    collections=["landsat-8-c2-l2"],
    bbox=geom.bounds,
    datetime="2015-01-01/2015-12-31"
)
items = list(search.get_items())

for item in items:
    sign(item)

# 懒加载并并行执行
ds = stac_load(items, bands=["red", "nir08"], chunks={"x": 2048, "y": 2048})

# 计算 NDVI
ndvi = (ds.nir08 - ds.red) / (ds.nir08 + ds.red)

# 并行计算 mean NDVI over AOI
mean_ndvi = ndvi.mean(dim=["x", "y"]).compute()
print(mean_ndvi)
